* https://diane-space.tistory.com/331
* https://data-science-hi.tistory.com/190

In [128]:
import numpy as np
import pandas as pd
import yfinance as yf

import torch
import torch.nn as nn
import torch.optim as optim

AMZN = yf.download('AMZN', start='2013-01-01', end='2019-12-31', progress=False)
all_data = AMZN[['Adj Close', 'Open', 'High', 'Low', "Close", "Volume"]].round(2)
print("There are " + str(all_data[:'2018'].shape[0]) + " observations in the training data")
print("There are " + str(all_data['2019':].shape[0]) + " observations in the test data")

#all_data['Adj Close'].plot()

There are 1510 observations in the training data
There are 251 observations in the test data


In [136]:
def split_data(data, steps: int = 5, features: int = 1, outputs: int = 2, debug: bool = False) -> tuple:
    debug and print("Split Data Source =", data)
    x, y = [], []
    for i in range(steps, len(data) - outputs + 1):
        x.append(data[i - steps:i])
        y.append(data[i:i + outputs, 0])
    return torch.FloatTensor(x), torch.FloatTensor(y)


def build_data(data, steps: int = 5, features: int = 1, outputs: int = 2, debug: bool = False) -> tuple:
    debug and print("Build Data Source =", data)
    return split_data(
        data[:'2018'].iloc[:, 0:features].values,
        steps, features, outputs, debug
    ) + split_data(
        data['2019':].iloc[:, 0:features].values,
        steps, features, outputs, debug
    )


debug = False
x_train, y_train, x_test, y_test = build_data(all_data[:10], features=1, outputs=1, debug=debug)
debug and print("x_train = ", x_train.shape, x_train)
debug and print("y_train = ", y_train.shape, y_train)
debug and print("x_test = ", x_test.shape, x_test)
debug and print("y_test = ", y_test.shape, y_test)

from torch.utils.data import TensorDataset, DataLoader

BATCH_SIZE = 2
train_data_loader = DataLoader(TensorDataset(x_train, y_train), batch_size=BATCH_SIZE, shuffle=False, drop_last=True)
test_data_loader = DataLoader(TensorDataset(x_test, y_test), batch_size=BATCH_SIZE, shuffle=False, drop_last=True)